In [ ]:
# !pip install selenium

In [ ]:
# !pip install pandas lxml beautifulsoup4 html5lib matplotlib --user

In [1]:
#Importing packages
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import re
import warnings


In [2]:
PolymersUpper = []
PolymersFull = []
Check = 'index/poly'

In [3]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

In [4]:
## ABIndex

driver.get('http://www.polymerdatabase.com/home.html')

AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'polymer index/')]")
for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
        PolymersUpper.append(Poly.get_attribute("href"))


In [5]:
## CDIndex

driver.get('http://www.polymerdatabase.com/polymer%20index/C-D%20index.html')


AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'poly')]")
for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
        PolymersUpper.append(Poly.get_attribute("href"))


In [6]:
## Adding one that didn't have "poly" in XPath.

PolymersUpper.append('http://www.polymerdatabase.com/polymer%20index/epoxyresins.html')


In [7]:
## EFIndex

driver.get('http://www.polymerdatabase.com/polymer%20index/E-F%20index.html')

AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'poly')]")

for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
         PolymersUpper.append(Poly.get_attribute("href"))


In [8]:
## Adding one that didn't have "poly" in XPath.

PolymersUpper.append('http://www.polymerdatabase.com/polymer%20index/Cellulose.html')

In [9]:
## GLIndex

driver.get('http://www.polymerdatabase.com/polymer%20index/G-L%20index.html')

AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'poly')]")

for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
         PolymersUpper.append(Poly.get_attribute("href"))


In [10]:
## MPIndex

driver.get('http://www.polymerdatabase.com/polymer%20index/M-P%20index.html')

AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'poly')]")

for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
         PolymersUpper.append(Poly.get_attribute("href"))


In [11]:
## SVIndex

driver.get('http://www.polymerdatabase.com/polymer%20index/S-V%20index.html') 

AllLinksPolymer = driver.find_elements_by_xpath("//*[contains(@href,'poly')]")

for Poly in AllLinksPolymer:
    if Check in Poly.get_attribute("href"):
         PolymersUpper.append(Poly.get_attribute("href"))


In [12]:
## Remove duplicates

PolymersUpper = list(dict.fromkeys(PolymersUpper))


In [13]:
## Go into each index link and grab all links

for i in PolymersUpper:
    driver.get(i)
    
    AllLinksPolymer = driver.find_elements_by_tag_name("a")
    for Poly in AllLinksPolymer:
        PolymersFull.append(Poly.get_attribute("href"))


In [14]:
## Remove duplicates

PolymersFinal = list(dict.fromkeys(PolymersFull))


In [15]:
## Filtering links I don't want.
Polymer = []

Filter1 = '/polymers/'

for i in PolymersFinal:
    if Filter1 in i:
        Polymer.append(i)

## Removing items that are formatted differently

Polymer.remove('http://www.polymerdatabase.com/polymers/polyvinylformal.html')
Polymer.remove('http://www.polymerdatabase.com/polymers/polyvinylbutyral.html')

## Come back and add these in manually once data is exported


In [16]:

## Instantiating first run to ensure correct format
dfi = pd.read_html('http://www.polymerdatabase.com/polymers/polyacrylamide.html')

dfPolymer = []

## df0 processing

df0 = dfi[0]
df0 = pd.DataFrame.transpose(dfi[0])
df0.columns = df0.iloc[0]
df0 = df0.drop(df0.index[[0]])
df0 = df0.drop(columns=['ACRONYMS'])

## df3 processing

df3 = pd.DataFrame.transpose(dfi[3])
df3 = df3.drop(df3.index[[1,3]])
as_list = df3.index.tolist()
idx = as_list.index(2)
as_list[idx] = 1
df3.index = as_list
df3.columns = df3.iloc[0]
df3 = df3.drop(df3.index[[0]])
df3 = df3.drop(columns=['PROPERTY'])

Columns = list(df3)

for i in Columns:
    test = str(df3[i].values)
    temp = re.findall(r"[-+]?\d*\.\d+|\d+", test) 
    res = list(map(float, temp)) 
    res = np.nanmean(res)
    df3[i] = res

## df1 processing

df1 = dfi[1]
df1 = pd.DataFrame.transpose(dfi[1])
df1.columns = df1.iloc[0]
df1 = df1.drop(df1.index[[0]])
df1 = df1.drop(columns=['STRUCTURE'])

## Combining df1, df3, and df0

dfCombinedi = pd.concat([df3, df1],axis=1)
dfCombinedi = pd.concat([df0, dfCombinedi],axis=1)

## Remove data to just have column headers

dfPolymer = dfCombinedi
dfPolymer = dfPolymer.drop(dfPolymer.index[[0]])


In [17]:
## Defining a function to process each page of polymer data

def ProcessData(dfs):
    ## df0 processing

    df0 = dfs[0]
    df0 = pd.DataFrame.transpose(dfs[0])
    df0.columns = df0.iloc[0]
    df0 = df0.drop(df0.index[[0]])
    df0 = df0.drop(columns=['ACRONYMS'])

    ## df3 processing

    df3 = pd.DataFrame.transpose(dfs[3])
    df3 = df3.drop(df3.index[[1,3]])
    as_list = df3.index.tolist()
    idx = as_list.index(2)
    as_list[idx] = 1
    df3.index = as_list
    df3.columns = df3.iloc[0]
    df3 = df3.drop(df3.index[[0]])
    df3 = df3.drop(columns=['PROPERTY'])

    Columns = list(df3)

    for i in Columns:
        test = str(df3[i].values)
        temp = re.findall(r"[-+]?\d*\.\d+|\d+", test) 
        res = list(map(float, temp)) 
        with warnings.catch_warnings():
            warnings.filterwarnings(action='ignore', message='Mean of empty slice')
            res = np.nanmean(res)
        df3[i] = res

    ## df1 processing

    df1 = dfs[1]
    df1 = pd.DataFrame.transpose(dfs[1])
    df1.columns = df1.iloc[0]
    df1 = df1.drop(df1.index[[0]])
    df1 = df1.drop(columns=['STRUCTURE'])

    ## Combining df1, df3, and df0

    dfCombined = pd.concat([df3, df1],axis=1)
    dfCombined = pd.concat([df0, dfCombined],axis=1)

    return dfCombined

    

In [18]:

## Go into each polymer link and grab all useful information

dfs = []

for i in Polymer:
    dfs = pd.read_html(i)
    dfPolymer = ProcessData(dfs).append(dfPolymer)
                                    

In [36]:
## Fixing some columns with the same title

cols = []
count = 1
for column in dfPolymer.columns:
    if column == 'CAS #':
        cols.append(f'CAS #_{count}')
        count+=1
        continue
    cols.append(column)
dfPolymer.columns = cols

dfPolymer = dfPolymer.rename(columns={'CAS #_1':'CAS #'})
dfPolymer = dfPolymer.rename(columns={'CAS #_2':'Monomer CAS'})

cols = []
count = 1
for column in dfPolymer.columns:
    if column == 'COMMON NAMES':
        cols.append(f'COMMON NAMES_{count}')
        count+=1
        continue
    cols.append(column)
dfPolymer.columns = cols

dfPolymer = dfPolymer.rename(columns={'COMMON NAMES_1':'COMMON NAMES'})
dfPolymer = dfPolymer.rename(columns={'COMMON NAMES_2':'Monomer Common Names'})

,POLYMER CLASS,COMMON NAMES,STRUCTURE BASED NAME,CAS #,CurlySMILES,Molecular Weight of Repeat unit,Van-der-Waals Volume VvW,Molar Volume Vm,Density ρ,Solubility Parameter δ,Molar Cohesive Energy Ecoh,Glass Transition Temperature Tg,Molar Heat Capacity Cp,Entanglement Molecular Weight Me,Index of Refraction n,Monomer CAS,Monomer Common Names,SMILES,Std. InChI,Std. InChIKey
1,Polyvinylsulfide,"Poly(propyl vinyl thioether), Poly(vinyl propy...",Poly(propylthioethylene),NaN,C{-}C{n+}(SCCC),102.20,61.30,99.90,1.025,17.60,30950.0,250.5,196.0,NaN,1.545,16330-21-5,"Propyl vinyl thioether, 1-(Vinylsulfanyl)propane",CCCSC=C,"1S/C5H10S/c1-3-5-6-4-2/h4H,2-3,5H2,1H3",HWRMMWBUMQCMJK-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(vinyl phenyl sulfide), Poly(phenylthioeth...",Poly[(vinylsulfanyl)benzene],NaN,C{-}C{n+}(Sc1ccccc1),136.22,76.21,115.70,1.180,19.85,45500.0,398.0,153.5,22800.0,1.655,1822-73-7,"Vinyl phenyl sulfide, Phenylthioethene, (Vinyl...",C=CSc1ccccc1,"1S/C8H8S/c1-2-9-8-6-4-3-5-7-8/h2-7H,1H2",GMPDOIGGGXSAPL-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(methyl vinyl thioether), Poly(vinyl methy...",Poly(methylthioethylene),NaN,C{-}C{n+}(SC),74.15,40.77,66.15,1.125,18.20,22150.0,269.5,135.5,7150.0,1.580,1822-74-8,"Methyl vinyl thioether, (Methylthio)ethene",CSC=C,"1S/C3H6S/c1-3-4-2/h3H,1H2,2H3",AMBKPYJJYUKNFI-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(ethyl vinyl thioether), Poly(vinyl ethyl ...",Poly(ethylthioethylene),NaN,C{-}C{n+}(SCC),88.17,51.05,82.10,1.075,17.85,26600.0,265.0,166.0,7900.0,1.555,627-50-9,"Ethyl vinyl thioether, (Ethylthio)ethene, Ethy...",CCSC=C,"1S/C4H8S/c1-3-5-4-2/h3H,1,4H2,2H3",AFGACPRTZOCNIW-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(butyl vinyl thioether), Poly(vinyl butyl ...",Poly(butylthioethylene),NaN,C{-}C{n+}(SCCCC),116.23,71.60,114.05,1.020,17.50,35450.0,259.5,225.5,14000.0,1.535,4789-70-2,Butyl vinyl thioether,CCCCSC=C,"1S/C6H12S/c1-3-5-6-7-4-2/h4H,2-3,5-6H2,1H3",LBMGSGLAWRZARD-UHFFFAOYSA-N


In [37]:
## Clarifying some of the column headers
dfPolymer = dfPolymer.rename(columns={'SMILES':'Monomer SMILES'})

## Adding in 2 polymers that couldn't be read earlier.
d = {'POLYMER CLASS':['Polyvinylether', 'Polyvinylether'], 'COMMON NAMES':['Poly(vinyl butyral),  (EXP DATA ONLY)', 'Poly(vinyl formal), (EXP DATA ONLY)'], 
     'STRUCTURE BASED NAME':['Poly[(2-propyl-1,3-dioxane-4,6-diyl)methylene]','Poly[(1,3-dioxane-4,6-diyl)methylene]'], 'CAS #':['63148-65-2','63450-15-7'], 
     'CurlySMILES':['C1{-}CC(OC(CCC)O1)C{n+}','C1{-}CC(OCO1)C{n+}'], 'Molecular Weight of Repeat unit':[141.21, 99.12], 
     'Van-der-Waals Volume VvW':[np.nan,np.nan], 'Molar Volume Vm':[125,79.85], 'Density ρ':[1.135,1.24], 'Solubility Parameter δ':[22.05,np.nan], 
     'Molar Cohesive Energy Ecoh':[63750,np.nan], 'Glass Transition Temperature Tg':[343,353], 'Molar Heat Capacity Cp':[np.nan,np.nan], 
     'Entanglement Molecular Weight Me':[np.nan,np.nan], 'Index of Refraction n':[1.495,1.49], 'Monomer CAS':['9002-89-5, 123-72-8','9002-89-5, 50-00-0'], 
     'Monomer Common Names':['Poly(vinylalcohol) + Butyraldehyde','Poly(vinylalcohol) + Formaldehyde (- H2O)'], 'Monomer SMILES':['C{-}C(O){n+}.O=CCCC','C{-}C(O){n+}.O=C'], 
     'Std. InChI':[np.nan,np.nan], 'Std. InChIKey':[np.nan,np.nan]}

dfAdd = pd.DataFrame(data=d)
dfPolymer = dfAdd.append(dfPolymer)

(248, 20)

In [38]:
dfPolymer.shape

(248, 20)

In [39]:
dfPolymer.head()

,POLYMER CLASS,COMMON NAMES,STRUCTURE BASED NAME,CAS #,CurlySMILES,Molecular Weight of Repeat unit,Van-der-Waals Volume VvW,Molar Volume Vm,Density ρ,Solubility Parameter δ,Molar Cohesive Energy Ecoh,Glass Transition Temperature Tg,Molar Heat Capacity Cp,Entanglement Molecular Weight Me,Index of Refraction n,Monomer CAS,Monomer Common Names,Monomer SMILES,Std. InChI,Std. InChIKey
0,Polyvinylether,"Poly(vinyl butyral), (EXP DATA ONLY)","Poly[(2-propyl-1,3-dioxane-4,6-diyl)methylene]",63148-65-2,C1{-}CC(OC(CCC)O1)C{n+},141.21,NaN,125.00,1.135,22.05,63750.0,343.0,NaN,NaN,1.495,"9002-89-5, 123-72-8",Poly(vinylalcohol) + Butyraldehyde,C{-}C(O){n+}.O=CCCC,NaN,NaN
1,Polyvinylether,"Poly(vinyl formal), (EXP DATA ONLY)","Poly[(1,3-dioxane-4,6-diyl)methylene]",63450-15-7,C1{-}CC(OCO1)C{n+},99.12,NaN,79.85,1.240,NaN,NaN,353.0,NaN,NaN,1.490,"9002-89-5, 50-00-0",Poly(vinylalcohol) + Formaldehyde (- H2O),C{-}C(O){n+}.O=C,NaN,NaN
1,Polyvinylsulfide,"Poly(propyl vinyl thioether), Poly(vinyl propy...",Poly(propylthioethylene),NaN,C{-}C{n+}(SCCC),102.20,61.30,99.90,1.025,17.60,30950.0,250.5,196.0,NaN,1.545,16330-21-5,"Propyl vinyl thioether, 1-(Vinylsulfanyl)propane",CCCSC=C,"1S/C5H10S/c1-3-5-6-4-2/h4H,2-3,5H2,1H3",HWRMMWBUMQCMJK-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(vinyl phenyl sulfide), Poly(phenylthioeth...",Poly[(vinylsulfanyl)benzene],NaN,C{-}C{n+}(Sc1ccccc1),136.22,76.21,115.70,1.180,19.85,45500.0,398.0,153.5,22800.0,1.655,1822-73-7,"Vinyl phenyl sulfide, Phenylthioethene, (Vinyl...",C=CSc1ccccc1,"1S/C8H8S/c1-2-9-8-6-4-3-5-7-8/h2-7H,1H2",GMPDOIGGGXSAPL-UHFFFAOYSA-N
1,Polyvinylsulfide,"Poly(methyl vinyl thioether), Poly(vinyl methy...",Poly(methylthioethylene),NaN,C{-}C{n+}(SC),74.15,40.77,66.15,1.125,18.20,22150.0,269.5,135.5,7150.0,1.580,1822-74-8,"Methyl vinyl thioether, (Methylthio)ethene",CSC=C,"1S/C3H6S/c1-3-4-2/h3H,1H2,2H3",AMBKPYJJYUKNFI-UHFFFAOYSA-N


In [40]:
dfPolymer.tail()

,POLYMER CLASS,COMMON NAMES,STRUCTURE BASED NAME,CAS #,CurlySMILES,Molecular Weight of Repeat unit,Van-der-Waals Volume VvW,Molar Volume Vm,Density ρ,Solubility Parameter δ,Molar Cohesive Energy Ecoh,Glass Transition Temperature Tg,Molar Heat Capacity Cp,Entanglement Molecular Weight Me,Index of Refraction n,Monomer CAS,Monomer Common Names,Monomer SMILES,Std. InChI,Std. InChIKey
1,Polyacrylamide,Poly(N-phenylacrylamide),Poly[(N-phenylaminocarbonyl)ethylene],NaN,C{-}C{n+}(C(=O)Nc1ccccc1),147.18,83.71,124.30,1.185,25.85,81800.0,498.0,177.0,46000.0,1.615,2210-24-4,N-Phenylacrylamide,C=CC(=O)Nc1ccccc1,"1S/C9H9NO/c1-2-9(11)10-8-6-4-3-5-7-8/h2-7H,1H2...",BPCNEKWROYSOLT-UHFFFAOYSA-N
1,Polyacrylamide,"Poly(N-tert-butyl acrylamide), Poly[N-(2-methy...",Poly[(N-tert-butylaminocarbonyl)ethylene],NaN,C{-}C{n+}(C(=O)NC(C)(C)C),127.19,79.10,122.65,1.040,22.45,62100.0,417.5,192.0,28250.0,1.505,107-58-4,N-tert-Butyl acrylamide,C=CC(=O)NC(C)(C)C,"1S/C7H13NO/c1-5-6(9)8-7(2,3)4/h5H,1H2,2-4H3,(H...",XFHJDMUEHUHAJW-UHFFFAOYSA-N
1,Polyacrylamide,Poly(N-octyl acrylamide),Poly[(N-octylaminocarbonyl)ethylene],NaN,C{-}C{n+}(C(=O)NCCCCCCCC),183.30,120.22,186.55,0.985,20.95,82550.0,228.0,382.5,54250.0,1.485,10124-68-2,NaN,CCCCCCCCNC(=O)C=C,1S/C11H21NO/c1-3-5-6-7-8-9-10-12-11(13)4-2/h4H...,AWGZKFQMWZYCHF-UHFFFAOYSA-N
1,Polyacrylamide,Poly(N-isopropyl acrylamide),Poly[(N-isopropylaminocarbonyl)ethylene],25189-55-3,C{-}C{n+}(C(=O)NC(C)C),113.16,68.83,107.35,1.055,23.50,59650.0,402.5,169.0,23000.0,1.500,2210-25-5,N-Isopropyl acrylamide,C=CC(=O)NC(C)C,"InChI=1S/C6H11NO/c1-4-6(8)7-5(2)3/h4-5H,1H2,2-...",QNILTEGFHQSKFF-UHFFFAOYSA-N
1,Polyacrylamide,Poly(acrylamide),"Poly(2-propenimidic acid), Poly(2-propenamide)",9003-05-8,C{-}C{n+}(C(=O)N),71.08,38.60,59.40,1.200,25.90,40000.0,436.0,97.0,16000.0,1.490,79-06-1,Acrylamide,OC(=N)C=C,"1S/C3H5NO/c1-2-3(4)5/h2H,1H2,(H2,4,5)",HRPVXLWXLXDGHG-UHFFFAOYSA-N


In [ ]:
dfPolymer['Glass Transition Temperature Tg'].iloc[54] = 378
dfPolymer['Glass Transition Temperature Tg'].iloc[179] = 207
dfPolymer['Molar Heat Capacity Cp'].iloc[22] = 109
dfPolymer['Density ρ'].iloc[77] = 2.21
dfPolymer['Density ρ'].iloc[85] = 1.22
dfPolymer['Density ρ'].iloc[183] = 1.12
dfPolymer['Index of Refraction n'].iloc[90] = 1.56

In [41]:
dfPolymer.to_csv('PolymerData.csv')